# 01 Alchemical free energy setup


<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Author: 
[Antonia Mey -- @ppxasjsm](https://github.com/ppxasjsm)   
[Lester Hedges -- @lohedges](https://github.com/lohedges)

## Learning objectives:
- Setup an alchemical solvation free energy simulation using BioSimSpace and SOMD
- Setup an alchemical solvation free energy simulation using BioSimSpace and Gromacs
- Setup an alchemical binding free energy simulation for Gromacs and SOMD using BioSimSpace

You will be using the following functions in BioSimSpace:

- `BSS.IO.ReadMolecules()` to load you molecules
- `BSS.Parameters.parameterise()` will be used to parametrise your molecules
- `BSS.Align.matchAtoms()` MCS matches atoms for the morphing
- `BSS.Align.rmsdAlign()` Aligns the molecules to be morphed
- `BSS.Align.merge()` Creates a merged molecule used for alchemical simulations
- `BSS.Solvent.tip3p()` Solvates a molecule in a tip3 water box
- `BSS.Protocol.FreeEnergy()` Defines the free energy protocol
- `BSS.FreeEnergy.Solvation()` Sets up a solvation free energy process
- `BSS.FreeEnergy.Binding()` Sets up a binding free energy process


**Reading time**:
~ 30 mins

**Jupyter cheat sheet**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;
- you can find the full documentation at [PyEMMA.org](http://www.pyemma.org).

## Table of Contents
1. [Working with ligands](#lig)    
   1.1 [Loading ligands](#load)   
   1.2 [parametrising ligands](#param)   
   1.3 [Minimisation and equilibration](#min)   
2. [Morphing ligands](#dtraj)   
   2.1 [MCS](#mcs)   
   2.2 [morphed ligand](#morph)   
3. [Running solvation free energy simulation](#solv)
4. [Binding free energy simulation](#bind)   
5. [Exercises](#exerc)   

#### Let's get all the necessary imports out of the way

In [1]:
%pylab inline
import BioSimSpace as BSS

Populating the interactive namespace from numpy and matplotlib


## 1. Free energy of solvation of ethane and methanol
<a id="lig"></a>


And some bla here

### 1.1 Loading ligands

In [2]:
# We assume the molecules to perturb are the first molecules in each system
ethane = BSS.IO.readMolecules('data/ethane.pdb').getMolecules()[0]
methanol = BSS.IO.readMolecules('data/methanol.pdb').getMolecules()[0]

In [3]:
BSS.viewMolecules('data/ethane.pdb')

Reading molecules from '['data/ethane.pdb']'
Rendering the molecules...


NGLWidget()

In [4]:
BSS.viewMolecules('data/methanol.pdb')

Reading molecules from '['data/methanol.pdb']'
Rendering the molecules...


NGLWidget()

### 1.2 Parametrising molecules

In [5]:
ethane = BSS.Parameters.parameterise(ethane, forcefield="gaff").getMolecule()
methanol = BSS.Parameters.parameterise(methanol, forcefield="gaff").getMolecule()

## 2. Creating merged system

### 2.1 MCS

In [6]:
mapping = {}
mapping = BSS.Align.matchAtoms(ethane, methanol)

# Align lig0 to lig1 based on the mapping.
ethane = BSS.Align.rmsdAlign(ethane, methanol, mapping)

# Merge the two ligands based on the mapping.
merged = BSS.Align.merge(ethane, methanol, mapping)



### 2.2 Creating a morph

In [7]:
merged._toPertFile('ethane_methanol.pert')

Molecule( 7 version 597 : nAtoms() = 8, nResidues() = 1 )

### 2.3 Solvation

In [8]:
solvated = BSS.Solvent.tip3p(molecule=merged, box=3*[40*BSS.Units.Length.angstrom])

## 3. Solvation free energy

In [9]:
# Create the free energy protocol.
protocol = BSS.Protocol.FreeEnergy(timestep=2*BSS.Units.Time.femtosecond, runtime=1*BSS.Units.Time.nanosecond, num_lam=9)


In [10]:
freenrg = BSS.FreeEnergy.Solvation(solvated, protocol, work_dir="ethane_methanol_somd", engine="SOMD")


In [ ]:
freenrg.run()

In [ ]:
## 4. Free energy of binding